<a href="https://colab.research.google.com/github/Ibraheem101/Data-Science-learning/blob/main/huggingface/2_hf_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizers